In [ ]:
import pandas as pd
import requests
from datasets import Dataset, Audio
import os, requests, pandas as pd
from urllib.parse import urlparse, unquote
from datasets import Dataset, Audio,load_dataset
from tqdm import tqdm
import whisper
import librosa
import json
import torch
from tqdm import tqdm
from transformers import WhisperForConditionalGeneration, Seq2SeqTrainer, Seq2SeqTrainingArguments, WhisperProcessor
from torch.nn.utils.rnn import pad_sequence
from transformers import get_scheduler
from torch.utils.data import Dataset, DataLoader
from transformers import WhisperForConditionalGeneration, WhisperTokenizer, WhisperFeatureExtractor, Trainer, TrainingArguments


In [ ]:
!pip install -U openai-whisper
!pip install -U openai-whisper
!apt-get install -y ffmpeg


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.2/803.2 kB 12.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for openai-whisper: filename=openai_whisper-20250625-py3-none-any.whl size=803979 sha256=ed8251e38718f21c430f7cc9c09a23a4902501b6e0c40e88da145a3694bbf64f
  Stored in directory: /root/.cache/pip/wheels/61/d2/20/09ec9bef734d126cba375b15898010b6cc28578d8afdde5869
Successfully built openai-whisper
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


In [ ]:
!pip install torchcodec

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 16.0 MB/s eta 0:00:00


In [ ]:
!conda install -c conda-forge ffmpeg


/bin/bash: line 1: conda: command not found


In [ ]:
!wget https://johnvansickle.com/ffmpeg/releases/ffmpeg-release-amd64-static.tar.xz -O ffmpeg-static.tar.xz


--2025-09-06 13:56:30--  https://johnvansickle.com/ffmpeg/releases/ffmpeg-release-amd64-static.tar.xz
Resolving johnvansickle.com (johnvansickle.com)... 107.180.57.212
Connecting to johnvansickle.com (johnvansickle.com)|107.180.57.212|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 41888096 (40M) [application/x-xz]
Saving to: ‘ffmpeg-static.tar.xz’

ffmpeg-static.tar.x 100%[===================>]  39.95M  33.2MB/s    in 1.2s    

2025-09-06 13:56:31 (33.2 MB/s) - ‘ffmpeg-static.tar.xz’ saved [41888096/41888096]



In [ ]:
!tar -xf ffmpeg-static.tar.xz
!./ffmpeg-*-amd64-static/ffmpeg -version

ffmpeg version 7.0.2-static https://johnvansickle.com/ffmpeg/  Copyright (c) 2000-2024 the FFmpeg developers
built with gcc 8 (Debian 8.3.0-6)
configuration: --enable-gpl --enable-version3 --enable-static --disable-debug --disable-ffplay --disable-indev=sndio --disable-outdev=sndio --cc=gcc --enable-fontconfig --enable-frei0r --enable-gnutls --enable-gmp --enable-libgme --enable-gray --enable-libaom --enable-libfribidi --enable-libass --enable-libvmaf --enable-libfreetype --enable-libmp3lame --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-librubberband --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libvorbis --enable-libopus --enable-libtheora --enable-libvidstab --enable-libvo-amrwbenc --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libdav1d --enable-libxvid --enable-libzvbi --enable-libzimg
libavutil      59.  8.100 / 59.  8.100
libavcodec     61.  3.100 / 61.  3.100
libavformat    61.  1.10

In [ ]:
!apt-get update
!apt-get install -y ffmpeg
!ffmpeg -version


Hit:1 https://cli.github.com/packages stable InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:5 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:6 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [80.4 kB]
Get:7 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease [24.3 kB]
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [2,002 kB]
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:10 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [9,241 kB]
Get:11 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:12 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,791 kB]
Get:13 https://ppa.lau

In [ ]:
df = pd.read_excel("/content/FT Data.xlsx")

In [ ]:
EXCEL_PATH = "/mnt/data/FT Data.xlsx"
audio_dir    = "/content/audio"
OUT_DS_DIR  = "/content/whisper_dataset_local"

In [ ]:
os.makedirs(audio_dir, exist_ok=True)


In [ ]:
tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-small", language="hindi", task="transcribe")
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")
processor = WhisperProcessor.from_pretrained("openai/whisper-small", language="hindi", task="transcribe")


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/967M [00:00<?, ?B/s]

generation_config.json: 0.00B [00:00, ?B/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

preprocessor_config.json: 0.00B [00:00, ?B/s]

In [ ]:
def download_file_if_missing(url, filepath):
    if not os.path.exists(filepath):
        response = requests.get(url)
        response.raise_for_status()
        with open(filepath, 'wb') as f:
            f.write(response.content)

In [ ]:
def load_transcription(transcription_url):
    response = requests.get(transcription_url)
    response.raise_for_status()
    data = response.json()
    # If JSON is list of segments, concatenate text fields
    if isinstance(data, list):
        return " ".join(segment.get('text', '') for segment in data).strip()
    # If JSON is a dict with 'text' field
    elif isinstance(data, dict):
        return data.get('text', '').strip()
    else:
        return ""


In [ ]:
dataset = {'audio': [], 'text': [], 'info': []}

In [ ]:
for idx, row in tqdm(df.iterrows(), total=len(df)):
    audio_url = row['rec_url_gcp']
    transcription_url = row['transcription_url_gcp']

    filename = os.path.basename(audio_url)
    audio_path = os.path.join(audio_dir, filename)


    download_file_if_missing(audio_url, audio_path)


    audio, sr = librosa.load(audio_path, sr=16000)
    transcription_text = load_transcription(transcription_url)
    info = {
        'user_id': row['user_id'],
        'recording_id': row['recording_id'],
        'language': row['language'],
        'duration': row['duration'],
        'metadata_url_gcp': row['metadata_url_gcp']
    }

    dataset['audio'].append(audio)
    dataset['text'].append(transcription_text)
    dataset['info'].append(info)

100%|██████████| 104/104 [04:00<00:00,  2.32s/it]


In [ ]:
dataset_df = pd.DataFrame(dataset)

In [ ]:
dataset_df.to_pickle('/content/drive/MyDrive/joshtalk_content/ft_dataset.pkl')

In [ ]:
dataset_df=pd.read_pickle('/content/drive/MyDrive/joshtalk_content/ft_dataset.pkl')

In [ ]:
dataset_df.head()

,audio,text,info
0,"[5.660588e-05, 0.000104444494, 6.564668e-05, 3...",अब काफी अच्छा होता है क्योंकि उनकी जनसंख्या बह...,"{'user_id': 245746, 'recording_id': 825780, 'l..."
1,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",जी जी जी जी जी । जी जी जी हां उधर हां जी हा हा...,"{'user_id': 291038, 'recording_id': 825727, 'l..."
2,"[5.3611493e-05, 4.5976667e-05, 3.947672e-05, 6...",लेकिन हम लोग इसे छुपछुप के लोगों के घर जाकर खे...,"{'user_id': 246004, 'recording_id': 988596, 'l..."
3,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",जी जी जी जी जी मेरे तो जैसे बहुत सारी यादे हैं...,"{'user_id': 93626, 'recording_id': 990175, 'la..."
4,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",हां जी पहले बात करते हैं विवाह की तो इस मुवी म...,"{'user_id': 286851, 'recording_id': 526266, 'l..."
